## Get Sizing for 5G Speed Tiers Propensity for various segments

In [ ]:
import pandas as pd
import numpy as np

### Ask from Chris -> 5G Dwell 

In [ ]:
%%bigquery df

SELECT 
    A.*
FROM `roaming-pr-66a1b0.5g_upsell.bq_5g_3500_zone_subs` A



In [ ]:
df

In [ ]:
%%bigquery prop_5g_df 
SELECT * FROM `cto-wln-sa-data-pr-bb5283.customer_personas_reports.speed_tiers_campaign_output_5G` B  

In [ ]:
prop_5g_df

In [ ]:
merged_df = pd.merge(df, prop_5g_df[['MSISDN', 'propensity_score']], how='inner', left_on='msisdn_num', right_on = 'MSISDN')

In [ ]:
merged_df

In [ ]:
%%bigquery plan_5g_df 

SELECT 
    pi_prod_instnc_resrc_str AS MSISDN,
    pp_recur_chrg_amt,
    pi_prim_pp_sls_start_ts,
    pp_catlg_itm_nm, 
    CASE 
        WHEN pp_catlg_itm_nm LIKE '%5G+%' THEN 'Yes'
        WHEN pp_catlg_itm_nm LIKE '%5G %' THEN 'Yes'
        ELSE 'No'
    END AS plan_5G_5G_plus

FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` 
WHERE 
  prod_instnc_ts = (SELECT MAX(prod_instnc_ts) FROM `cio-datahub-enterprise-pr-183a.ent_cust_cust.bq_prod_instnc_snpsht` )  -- Get most recent date in snapshot table

Exclude customers with 5G Plan

In [ ]:
plan_5g_df.loc[plan_5g_df.plan_5G_5G_plus=='No'].reset_index(drop=True).head()

In [ ]:
merged_df_2 = merged_df.loc[merged_df.msisdn_num.isin(plan_5g_df.loc[plan_5g_df.plan_5G_5G_plus=='No'].reset_index(drop=True)['MSISDN'])]

531k customers identified, split between 5G capable = True and False -> Figure out how many are high and how many are low propensity

In [ ]:
merged_df_2

In [ ]:
merged_df_2.groupby(['is_5g_capable'])[['propensity_score']].describe()

5G Customers in 3.5GHz area not on 5G Plan

In [ ]:
merged_df_2.loc[(merged_df_2.is_5g_capable ==True) & (merged_df_2.propensity_score >= merged_df_2.propensity_score.quantile(0.5))].shape

4G Customers in 3.5GHz area not on 5G Plan

In [ ]:
merged_df_2.loc[(merged_df_2.is_5g_capable ==False) & (merged_df_2.propensity_score >= merged_df_2.propensity_score.quantile(0.5))].shape

## Bell Zones

In [ ]:
%%bigquery bell_df

SELECT * FROM `roaming-pr-66a1b0.5g_upsell.bq_5g_3500_bell_zone_subs`

In [ ]:
bell_df

In [ ]:
bell_df.groupby(['billg_addr_city_nm'])[['msisdn_num']].count().reset_index().sort_values(by='msisdn_num', ascending=False)

In [ ]:
merged_bell_df = pd.merge(bell_df, prop_5g_df[['MSISDN', 'propensity_score']], how='inner', left_on='msisdn_num', right_on = 'MSISDN')

In [ ]:
merged_bell_df.groupby(['billg_addr_city_nm'])[['MSISDN']].count().reset_index().sort_values(by='MSISDN', ascending=False)

In [ ]:
merged_bell_df

Exclude customers on 5G Plan

In [ ]:

merged_bell_df_2 = merged_bell_df.loc[merged_bell_df.msisdn_num.isin(plan_5g_df.loc[plan_5g_df.plan_5G_5G_plus=='No'].reset_index(drop=True)['MSISDN'])]

In [ ]:
merged_bell_df_2

In [ ]:
merged_bell_df_2.loc[(merged_bell_df_2.is_5g_capable ==True) & (merged_bell_df_2.propensity_score >= merged_bell_df_2.propensity_score.quantile(0.5))].shape

In [ ]:
merged_bell_df_2.loc[(merged_bell_df_2.is_5g_capable ==False) & (merged_bell_df_2.propensity_score >= merged_bell_df_2.propensity_score.quantile(0.5))].shape